In [12]:
# Useful library for embeddings

import class_lstm
from class_lstm import MimicLSTM

import word2vec_preprocess
import word2vec_preprocess as preprocess

import gensim
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import importlib

from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing import sequence 
import tensorflow as tf

import gensim

#word2vec = gensim.models.KeyedVectors.load_word2vec_format(
#    './word2vec_model/GoogleNews-vectors-negative300.bin', binary=True) 


In [2]:
def nn_word(word, word_emb_dict, lstm_model, char_dict, word2vec):
    indices = np.array([char_dict[letter] for letter in word])
    indices = sequence.pad_sequences([indices], maxlen=25) 
    prediction = lstm_model.predict(tf.reshape(indices,(1,-1)), steps=1)
    
    actual = np.reshape(word2vec[word],(1,-1))
    actual_sim = cosine_similarity(actual,prediction)
    print("Actual sim is %s" %actual_sim)
    
    best_word = ''
    best_sim = 0
    total_sim = 0
    total_count = 0
    
    for k,v in word_emb_dict.items():
        
        v = np.reshape(v,(1,-1))
        sim = cosine_similarity(prediction, v)
        total_sim += sim
        total_count += 1
        
        if sim > best_sim:
            best_word = k
            best_sim = sim
            print(best_word, best_sim)
    avg_sim = total_sim/total_count
    
    print("actual sim was %s while avg was %s" %(actual_sim,avg_sim))
    return actual_sim, avg_sim

In [3]:
data_dict, chardict = preprocess.run()

In [4]:
layers = 1
H = 50
character_dim = 20
output_dim = 300

In [13]:
# Reload our packages as we test 
importlib.reload(class_lstm)
importlib.reload(word2vec_preprocess)

<module 'word2vec_preprocess' from '/home/cgleach/w266/final_project/w266-final-project-OOV-fake-bias-detection/original_work/word2vec_preprocess.py'>

In [17]:
model = MimicLSTM(layers, H, chardict, character_dim, data_dict, 
                  batch_size=5000, loss_function='custom_cosine')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 25, 20)            1120      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100)               28400     
_________________________________________________________________
dense_4 (Dense)              (None, 300)               30300     
Total params: 59,820
Trainable params: 59,820
Non-trainable params: 0
_________________________________________________________________
Train on 684279 samples, validate on 76031 samples
Epoch 1/5
595000/684279 [=========================>....] - ETA: 12s - loss: 0.9990

KeyboardInterrupt: 

In [ ]:
a,b = nn_word("Vercellotti", data_dict, model.lstm, chardict, word2vec)


In [18]:
layers = 2
H = 300
character_dim = 300
output_dim = 300

In [ ]:
model_complex = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 300)           16800     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 25, 600)           1442400   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 600)               2162400   
_________________________________________________________________
dense_5 (Dense)              (None, 300)               180300    
Total params: 3,801,900
Trainable params: 3,801,900
Non-trainable params: 0
_________________________________________________________________
Train on 684279 samples, validate on 76031 samples
Epoch 1/5
 20000/684279 [..............................] - ETA: 1:07:38 - loss: 0.9990